In [7]:
import re
import nltk
import nturl2path

import pandas as pd
import numpy as np

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for Lemmatization
import spacy

# plotting
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline



In [10]:
df = pd.read_csv('data/bbc-text.csv')
df.shape


(2225, 2)

In [11]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [14]:
# convert to list
data = df.text.values.tolist()

# remove emails
data = [re.sub('\S*@\S*\s?', '',i) for i in data]

# remove new line characters
data = [re.sub('\s+', ' ', i) for i in data]

# remove distracting single quotes
data = [re.sub("\'", "", i) for i in data]

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_39144/3177382541.py:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '',i) for i in data]
/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_39144/3177382541.py:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', i) for i in data]


In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
print(data_words[:1])

[['tv', 'future', 'in', 'the', 'hands', 'of', 'viewers', 'with', 'home', 'theatre', 'systems', 'plasma', 'high', 'definition', 'tvs', 'and', 'digital', 'video', 'recorders', 'moving', 'into', 'the', 'living', 'room', 'the', 'way', 'people', 'watch', 'tv', 'will', 'be', 'radically', 'different', 'in', 'five', 'years', 'time', 'that', 'is', 'according', 'to', 'an', 'expert', 'panel', 'which', 'gathered', 'at', 'the', 'annual', 'consumer', 'electronics', 'show', 'in', 'las', 'vegas', 'to', 'discuss', 'how', 'these', 'new', 'technologies', 'will', 'impact', 'one', 'of', 'our', 'favourite', 'pastimes', 'with', 'the', 'us', 'leading', 'the', 'trend', 'programmes', 'and', 'other', 'content', 'will', 'be', 'delivered', 'to', 'viewers', 'via', 'home', 'networks', 'through', 'cable', 'satellite', 'telecoms', 'companies', 'and', 'broadband', 'service', 'providers', 'to', 'front', 'rooms', 'and', 'portable', 'devices', 'one', 'of', 'the', 'most', 'talked', 'about', 'technologies', 'of', 'ces', 'ha

In [19]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]



/Users/niclapenna/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:873: UserWarning: [W094] Model 'en_core_web_sm' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.0,<3.5.0
  warnings.warn(warn_msg)


In [33]:
# spacy lemmetization
import spacy
import en_core_web_sm

nlp=en_core_web_sm.load()

def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [34]:
# remove stop words
data_words_nostops = remove_stopwords(data_words)

# do lemmatization keeping only noun
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN'])

In [35]:
data_lemmatized

[['tv',
  'viewer',
  'home',
  'theatre',
  'system',
  'definition',
  'video',
  'recorder',
  'living',
  'room',
  'way',
  'people',
  'tv',
  'year',
  'time',
  'expert',
  'panel',
  'consumer',
  'electronic',
  'show',
  'technology',
  'impact',
  'pastime',
  'trend',
  'programme',
  'content',
  'viewer',
  'home',
  'network',
  'cable',
  'satellite',
  'telecom',
  'company',
  'service',
  'provider',
  'room',
  'device',
  'technology',
  'video',
  'recorder',
  'dvr',
  'box',
  'sky',
  'system',
  'people',
  'record',
  'store',
  'pause',
  'wind',
  'tv',
  'programme',
  'technology',
  'tv',
  'definition',
  'tv',
  'business',
  'definition',
  'programming',
  'people',
  'wind',
  'advert',
  'network',
  'channel',
  'schedule',
  'carte',
  'network',
  'cable',
  'satellite',
  'company',
  'term',
  'advertising',
  'revenue',
  'brand',
  'identity',
  'viewer',
  'loyalty',
  'channel',
  'technology',
  'moment',
  'concern',
  'uptake',
  'serv

In [36]:
# create dictionary, needed as input for LDA alongside corpus
id2word = corpora.Dictionary(data_lemmatized)
print(id2word)
# create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

Dictionary<10584 unique tokens: ['advert', 'advertising', 'bill', 'book', 'box']...>


In [39]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            random_state=100,
                                            chunksize=200,
                                            passes=10,
                                            per_word_topics=True)

In [42]:
import pprint
# Print the Keyword in the 5 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"player" + 0.021*"year" + 0.020*"game" + 0.019*"time" + 0.016*"team" '
  '+ 0.013*"club" + 0.012*"side" + 0.011*"match" + 0.010*"week" + '
  '0.010*"manager"'),
 (1,
  '0.039*"film" + 0.021*"year" + 0.018*"music" + 0.015*"award" + '
  '0.013*"domain" + 0.011*"album" + 0.010*"number" + 0.010*"show" + '
  '0.009*"band" + 0.009*"song"'),
 (2,
  '0.032*"game" + 0.011*"woman" + 0.010*"prisoner" + 0.009*"year" + '
  '0.009*"time" + 0.008*"world" + 0.008*"case" + 0.007*"man" + 0.007*"school" '
  '+ 0.007*"arm"'),
 (3,
  '0.025*"people" + 0.012*"phone" + 0.012*"attack" + 0.012*"technology" + '
  '0.011*"service" + 0.011*"computer" + 0.010*"network" + 0.010*"site" + '
  '0.009*"software" + 0.009*"user"'),
 (4,
  '0.020*"year" + 0.015*"government" + 0.012*"sale" + 0.010*"party" + '
  '0.009*"election" + 0.009*"company" + 0.009*"people" + 0.008*"country" + '
  '0.008*"plan" + 0.007*"labour"')]


In [44]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4498515665457409
